In [1]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import time as time

% matplotlib inline

In [4]:
""" RK 4 """
def rk4(f, x0, y0, x1, n):
    vx = [0] * (n + 1)
    vy = [0] * (n + 1)
    h = (x1 - x0) / float(n)
    vx[0] = x = x0
    vy[0] = y = y0
    for i in range(1, n + 1):
        k1 = h * f(x, y)
        k2 = h * f(x + 0.5 * h, y + 0.5 * k1)
        k3 = h * f(x + 0.5 * h, y + 0.5 * k2)
        k4 = h * f(x + h, y + k3)
        vx[i] = x = x0 + i * h
        vy[i] = y = y + (k1 + k2 + k2 + k3 + k3 + k4) / 6
    return vx, vy

def pNL_THz(omega,Eopt):
    size=Eopt.size
    pNL=np.empty(size)
    pNL[0]=np.dot(Eopt,Eopt)
    for i in range(1,size):
        pNL[i]=np.dot(Eopt[:-i],Eopt[i:])
    return 2*pNL*(omega[1]-omega[0])

# vx - an array for x variable, vy - an array for f(x), x0 - inital x, y0 - f(x0), x1 - end point for x, n - number of points

In [5]:
""" Some constants"""

deff = 68.5 #pm/V
epsilon0 = 8.85e-12 # F/m
#n_THz = 3.2# https://refractiveindex.info/?shelf=main&book=ZnTe&page=Marple
n_group = 3.0
bandwidth = 10e-6 # mm
c = 0.3
pi = np.pi
center_wavelength = 800e-6

"""Multiple THz frequency (0.1 ~ 300) -- for loop"""

OTHz = 0.1*2*pi
THzMax=80
size = 1000
OTHz = np.linspace(0.,THzMax,size)
num = 1000

# Define the ODE
omega0 = c*2*pi/center_wavelength
w = c*2*pi/(center_wavelength-bandwidth/2)-c*2*pi/(center_wavelength+bandwidth/2)
omega = np.linspace(omega0-THzMax/2,omega0+THzMax/2,size)

P = np.zeros([size,1])
K = np.zeros([size,1])
E = np.zeros([size, num])
Er = np.zeros([size, num])
Ei = np.zeros([size, num])

t0=time.time()

def optE(omega, omega0, w):
    return np.exp(-4*np.log(2)*((omega-omega0)/w)**2)
pNL = pNL_THz(omega,optE(omega,omega0,w))

for i in range(size):
    THz = OTHz[i]
    n_THz = np.exp(np.log(1.25)/75*(THz))+5.65
    #n_THz = 3.2
    #alpha = np.exp(6e-3*THz)+1
    alpha = 1
    delta_k = (THz/c)*(n_THz-n_group)
    #delta_k = 0
    def ODE(z, ETHz):
        return (-1j*THz*c/2/n_THz)*pNL[i]*np.exp(1j*delta_k*z) - 0.5*alpha*ETHz
    z, e = rk4(ODE, 0,0,8.5,num-1)
    
    Er[i,:] = np.real(e)
    Ei[i,:] = np.imag(e)
    E[i,:] = np.absolute(e)
#     P[i] = pNL
    K[i] = delta_k
print(time.time()-t0)

# Er - 2D array for real part, Ei - 2D array for imaginary part, E - 2D array for amplitude, P - 1D array for pNL, K - 1D array for delta_k

13.081485986709595
